In [1]:
import numpy as np
import pandas as pd
from pathmgmt import pathmgmt as myPath
import os

In [43]:
# Combine all alphas
def get_stat():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName or ('Lqd01' not in folderName and 'Lqd02' not in folderName):
            continue
        if 'Lqd010' in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'statistics'
        for fileName in os.listdir(folder):
            # print(fileName)
            if 'Rank' in fileName:
                continue
            file = folder/fileName
            df = pd.read_csv(file)
            df.index = [f'{folderName};{fileName[:-15]}']
            # print(df)
            if res.empty:
                res = df
            else:
                res = pd.concat([res, df])
    return res

In [44]:
stats = get_stat()

In [45]:
stats.describe()

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
count,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000
mean,0.160933,-0.010582,2.271416,-0.069722,0.008059,0.103944,0.228432,0.192778
std,0.219960,0.101104,1.966433,0.891732,0.008088,0.172197,0.215664,0.302858
min,-0.645902,-0.819293,-4.206263,-4.454130,-0.005901,0.002100,0.063337,0.008382
25%,0.015656,-0.040458,0.568014,-0.481123,0.001564,0.028088,0.136250,0.028254
50%,0.127074,0.009319,2.359203,0.098583,0.006624,0.071449,0.184498,0.073436
75%,0.237514,0.034107,3.677315,0.346202,0.013369,0.110496,0.260010,0.231282
max,1.297202,0.135350,7.926831,2.070061,0.038205,1.839375,2.336619,1.406306


In [48]:
good_alphas = list(stats.loc[(stats.IR > 3) & (stats['daily Turnover'] < 0.2)].index)

In [49]:
len(good_alphas)

79

In [50]:
# combine daily pnl
def get_pnl():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName or ('Lqd01' not in folderName and 'Lqd02' not in folderName):
            continue
        if 'Lqd010' in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'PnL_results'
        for fileName in os.listdir(folder):
            file = folder/fileName
            if 'Rank' in fileName:
                continue
            df = pd.read_csv(file)
            # df = df.set_index('time')
            # df = df['pnl']
            df = df[['time', 'pnl']]
            df.rename(
                columns={'pnl': f'{folderName};{fileName[:-13]}'}, inplace=True)
            if res.empty:
                res = df
            else:
                res = res.merge(df, on='time', how='outer')
    res = res.set_index('time').sort_index()
    return res


In [51]:
pnls = get_pnl()

In [52]:
corr = pnls[good_alphas].corr()

In [66]:
from itertools import combinations
import random
res = []
num = 5
good_candidates = list(
    ((corr.mask(np.eye(len(corr), dtype=bool)).abs() < 0.6).sum() > num).index)
random.shuffle(good_candidates)

In [67]:
len(good_candidates)

79

In [ ]:
for cols in combinations(good_candidates, num):
    corr_small = corr.loc[cols, cols]
    if (~(corr_small.mask(np.eye(len(corr_small), dtype=bool)).abs() > 0.6).any()).sum() == num:
        res.append(corr_small)
        # break

In [75]:
# res[0]


In [76]:
# stats.loc[stats.index.isin(res[0].index)]


In [77]:
# res[0]

In [78]:
# stats.loc[stats.index.isin(res[0].index)]

In [69]:
max_IR = float('-inf')
max_idx = -1
for i in range(len(res)):
    total_IR = np.sum([(stats.loc[stats.index==name]['IR long only']) for name in res[i].index])
    if total_IR > max_IR:
        max_IR = total_IR
        max_idx = i

In [70]:
res[max_idx]

,alpha.YaoLqd012-60days-zz9999;20180101-20201231-None-Industry-holding10days,alpha.YaoLqd022-5days-zz9999;20180101-20201231-None-Industry-holding10days,alpha.YaoLqd015-5days-zz9999;20180101-20201231-None-Industry-holding3days,alpha.YaoLqd020-20days-zz9999;20180101-20201231-None-Industry-holding10days,alpha.YaoLqd024-3days-zz9999;20180101-20201231-None-Industry-holding20days
alpha.YaoLqd012-60days-zz9999;20180101-20201231-None-Industry-holding10days,1.000000,0.576522,0.586043,0.229995,-0.036160
alpha.YaoLqd022-5days-zz9999;20180101-20201231-None-Industry-holding10days,0.576522,1.000000,0.237871,-0.103029,0.107829
alpha.YaoLqd015-5days-zz9999;20180101-20201231-None-Industry-holding3days,0.586043,0.237871,1.000000,0.588430,-0.330830
alpha.YaoLqd020-20days-zz9999;20180101-20201231-None-Industry-holding10days,0.229995,-0.103029,0.588430,1.000000,-0.497384
alpha.YaoLqd024-3days-zz9999;20180101-20201231-None-Industry-holding20days,-0.036160,0.107829,-0.330830,-0.497384,1.000000


In [71]:
stats.loc[stats.index.isin(res[max_idx].index)]

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
alpha.YaoLqd012-60days-zz9999;20180101-20201231-None-Industry-holding10days,0.294506,0.010195,4.963402,0.115113,0.027932,0.043530,0.147999,0.127935
alpha.YaoLqd015-5days-zz9999;20180101-20201231-None-Industry-holding3days,0.474005,0.019947,3.668362,0.196635,0.015559,0.103585,0.148279,0.171069
alpha.YaoLqd020-20days-zz9999;20180101-20201231-None-Industry-holding10days,0.200777,0.109144,3.232803,1.118589,0.008675,0.054788,0.137907,0.031080
alpha.YaoLqd022-5days-zz9999;20180101-20201231-None-Industry-holding10days,0.043953,0.025484,3.331949,0.167028,0.012234,0.014647,0.241737,0.135430
alpha.YaoLqd024-3days-zz9999;20180101-20201231-None-Industry-holding20days,0.317453,-0.087515,3.183296,-1.215493,0.007556,0.102125,0.290112,0.045109
